# PART 1

In [25]:
# Downloading csv file
! wget https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.csv

--2023-03-21 03:37:00--  https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174944099 (167M) [text/csv]
Saving to: ‘hdma-wi-2021.csv’

hdma-wi-2021.csv    100%[===================>] 166.84M  89.2MB/s    in 1.9s    

2023-03-21 03:37:02 (89.2 MB/s) - ‘hdma-wi-2021.csv’ saved [174944099/174944099]



In [26]:
# Uploading same file to HDFS twice
! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://main:9000/single.csv
! hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://main:9000/double.csv

In [27]:
! hdfs dfs -du -h hdfs://main:9000/

166.8 M  333.7 M  hdfs://main:9000/double.csv
166.8 M  166.8 M  hdfs://main:9000/single.csv


# PART 2

In [28]:
import requests

In [29]:
blocks = {}
total_blocks = 167
datanode1_blocks = 0
datanode2_blocks = 0
block_size = 1048576
offset = 0

resp = requests.get("http://main:9870/webhdfs/v1/single.csv?op=OPEN&offset={}".format(offset), allow_redirects=False)
location1 = resp.headers["Location"]
location1 = location1.split('?')[0]

while offset < (total_blocks * block_size):
    r = requests.get("http://main:9870/webhdfs/v1/single.csv?op=OPEN&offset={}".format(offset), allow_redirects=False)
    location = r.headers["Location"]
    location = location.split('?')[0]
    
    if location1 == location:
        datanode1_blocks += 1
        blocks[location1] = datanode1_blocks
    else:
        datanode2_blocks += 1
        blocks[location] = datanode2_blocks
    offset += block_size
blocks

{'http://e58547bc2648:9864/webhdfs/v1/single.csv': 80,
 'http://a0d1575c6b77:9864/webhdfs/v1/single.csv': 87}

# PART 3

In [30]:
BLOCK_SIZE = 1048576

In [32]:
import io
import time

sf_count = 0
mf_count = 0
sf2_count = 0
mf2_count = 0

class hdfsFile(io.RawIOBase):
    def __init__(self, path):
        resp = requests.get("http://main:9870/webhdfs/v1/{}?op=GETFILESTATUS".format(path))
        data = resp.json()
        length = data["FileStatus"]["length"]
        self.path = path
        self.offset = 0
        self.length = length

    def readable(self):
        return True

    def readinto(self, b):
        if self.offset < self.length:
            response = requests.get("http://main:9870/webhdfs/v1/{}?op=OPEN&offset={}&length={}".format(self.path, self.offset, len(b)))
            if response.status_code != 200:
                content = '\n'
                b[:len(content)] = bytes(content, "utf-8")
                self.offset += BLOCK_SIZE
                return 1
            else:
                txt = response.content
                b[:len(txt)] = txt
                self.offset += len(txt)
                return len(txt)
        else:
            return 0
        
# Buffer Size 1: 1 MB
start1 = time.time()
for line1 in io.BufferedReader(hdfsFile("single.csv"), buffer_size=1048576):  
    line1 = str(line1, "utf-8")
    if "Single Family" in line1:
        sf_count += 1
    elif "Multifamily" in line1:
        mf_count += 1
end1 = time.time()

time1 = end1 - start1

print("Counts from single.csv")
print("Single Family:", sf_count)
print("Multi Family:", mf_count)
print("Seconds:", time1)

print("\n")

# Buffer Size 2: 1/2 MB
start2 = time.time()
for line2 in io.BufferedReader(hdfsFile("single.csv"), buffer_size=524288):
    line2 = str(line2, "utf-8")
    if "Single Family" in line2:
        sf2_count += 1
    elif "Multifamily" in line2:
        mf2_count += 1
end2 = time.time()

time2 = end2 - start2
      
print("Counts from single.csv")
print("Single Family:", sf2_count)
print("Multi Family:", mf2_count)
print("Seconds:", time2)

Counts from single.csv
Single Family: 444874
Multi Family: 2493
Seconds: 4.200409173965454


Counts from single.csv
Single Family: 444874
Multi Family: 2493
Seconds: 7.3432700634002686


# PART 4

In [33]:
! hdfs dfsadmin -fs hdfs://main:9000/ -report

Configured Capacity: 10213466112 (9.51 GB)
Present Capacity: 1695715328 (1.58 GB)
DFS Remaining: 1433939968 (1.34 GB)
DFS Used: 261775360 (249.65 MB)
DFS Used%: 15.44%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 87
	Missing blocks (with replication factor 1): 87
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.2:9866 (project-3-anushkap3-worker-2.cs544net)
Hostname: e58547bc2648
Decommission Status : Normal
Configured Capacity: 10213466112 (9.51 GB)
DFS Used: 261775360 (249.65 MB)
Non DFS Used: 8500973568 (7.92 GB)
DFS Remaining: 1433939968 (1.34 GB)
DFS Used%: 2.56%
DFS Remaining%: 14.04%
Co

In [34]:
sf3_count = 0
mf3_count = 0
sf4_count = 0
mf4_count = 0

# Buffer Size 1: 1 MB - double.csv
for line3 in io.BufferedReader(hdfsFile("double.csv"), buffer_size=1048576):
    line3 = str(line3, "utf-8")
    if "Single Family" in line3:
        sf3_count += 1
    elif "Multifamily" in line3:
        mf3_count += 1

print("Counts from double.csv")
print("Single Family:", sf3_count)
print("Multi Family:", mf3_count)

print("\n")

# Buffer Size 1: 1 MB - single.csv 
for line4 in io.BufferedReader(hdfsFile("single.csv"), buffer_size=1048576):
    line4 = str(line4, "utf-8")
    if "Single Family" in line4:
        sf4_count += 1
    elif "Multifamily" in line4:
        mf4_count += 1

print("Counts from single.csv")
print("Single Family:", sf4_count)
print("Multi Family:", mf4_count)

Counts from double.csv
Single Family: 444874
Multi Family: 2493


Counts from single.csv
Single Family: 212700
Multi Family: 1343
